## Este es un ejemplo de uso de la clase BCIDataset para el TP3

El video donde es trabajado se encuentra en https://youtu.be/b0Z7m6-BrqY

In [1]:
from pathlib import Path
import utils
from typing import Callable
import scipy
import scipy.signal as sgn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def filtered_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 5
    f_high = 40
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx


def naif_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    fft = np.fft.rfft(signal_matrix)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx

In [3]:
class BCIDataset():
    def __init__(
        self, 
        csvs_path, 
        subject: str = 'all', 
        session: str = 'all',
        channel: str = 'all', 
        overlapping_fraction: float = 1/3, 
        window_size: int = 900,
        feature_extractor: Callable = naif_fft_features
    ):
        '''
        Object containing all examples from a time series from the dataset.
        Args:
            csvs_path (str): path al directorio donde estan los csv de los datos
            subject (str): sujeto a estudiar.
                Si se indica 'all' el dataset final generado tendrá ejemplos de todos los sujetos
            session (str): sesión a estudiar del sujeto seleccionado.
                si se indica 'all'  el dataset final generado tendrá ejemplos de todas las sesiones
            channel (str): 'ch0', 'ch1', 'ch2', 'ch3'. Si se indica 'all', los ejemplos serán la
                concatenación de los 4 canales.
            overlapping_fraction (float): porcentaje de desplazamiento de la "ventana" que hace el ejemplo
            window_size (int): tamaño de la ventana de tiempo que hace a un ejemplo (en muestras).
            feature_extractor (func): Función de extracción de features, le ingresa un arreglo
                (ejemplos en las filas, muestras en las columnas) y devuelve un arreglo (ejemplos en
                las filas y features en las columnas)
        '''
        self.csvs_path = Path(csvs_path)
        self.channel = channel
        self.parts = int(1 / overlapping_fraction)
        self.fraction = 1 / self.parts
        self.ws = window_size
        self.subject = subject
        self.session = session
        self.channels = ['ch0','ch1','ch2','ch3']
        self.feature_extractor = feature_extractor
        self.complete_dataset = utils.read_all_datasets(self.csvs_path)
        
        self.complete_examples_signal, self.complete_examples_features, \
        self.complete_labels, self.complete_metadata = \
            self.generate_examples()
        
    def generate_examples(self):
        # For each session generate the signal examples,
        # the feature extraction examples and labels arrays
        complete_examples_signal, complete_examples_features = [], []
        complete_labels, complete_metadata = [], []
        
        for subject in self.complete_dataset.subject.unique():
            # Just load the selected subject
            if self.subject != 'all' and subject != self.subject:
                continue
            sessions = \
                self.complete_dataset.loc[self.complete_dataset.subject==subject].session.unique()
            for session in sessions:
                # Just load the selected session
                if self.session != 'all' and session != self.session:
                    continue
                
                print(f'Processing subject: {subject} - session: {session}...')
                
                # Generate a subset of the dataset only with the desired rows
                selection = self.complete_dataset.loc[
                    (self.complete_dataset.subject==subject) &
                    (self.complete_dataset.session==session)
                ]
                
                # Standarize length of the array to a multiple to window size
                labels = selection.label.values
                n_rows = labels.shape[0] // self.ws
                labels = labels[: n_rows * self.ws]

                # Generate examples from the signal
                n_examples = n_rows * self.parts - (self.parts - 1)
                examples = np.empty((n_examples, self.ws))
                most_frec_labels = np.empty((n_examples, self.ws))
                times = np.empty((n_examples, self.ws))
                
                concat_ch_examples_signal, concat_ch_examples_features = [], []

                for k, ch in enumerate(self.channels):
                    # Use the four channels or just one
                    if self.channel != 'all' and self.channel != ch:
                        continue
                    
                    # Standarize length of the signal to a multiple to window size
                    signal = selection[ch].values
                    signal = signal[: n_rows * self.ws]
                    time = selection.time.values
                    time = time[: n_rows * self.ws]

                    # Increase the number of examples by overlapping the windows
                    for part in range(self.parts):
                        
                        # Find the place in the output array for each example
                        position = np.arange(part, n_examples, self.parts)
                        #position = position if part == 0 else position[:-part]

                        # Crop the signal according to the window size and overlap
                        start = int(self.ws / self.parts) * part
                        end = -int(self.ws - (self.ws / self.parts) * part)
                        end = end if part!=0 else signal.shape[0]
                        subset_signal = signal[start:end]
                        subset_labels = labels[start:end]
                        subset_times = time[start:end]

                        # Generate the examples
                        n_rows_ = int(subset_signal.shape[0]/self.ws)
                        examples[position, :] = subset_signal.reshape((n_rows_, self.ws))
                        most_frec_labels[position, :] = subset_labels.reshape((n_rows_, self.ws))
                        times[position, :] = subset_times.reshape((n_rows_, self.ws))
                    
                    # Obtain most frequent label
                    labels_ = scipy.stats.mode(most_frec_labels, axis=1).mode
                    labels_temp = scipy.stats.mode(most_frec_labels, axis=1).count
                    pureness = labels_temp == self.ws
                    # Get first and last time of the window
                    times_ = np.asarray([np.min(times, axis=1), np.max(times, axis=1)]).T
                    # Extract features
                    features = self.feature_extractor(examples)

                    concat_ch_examples_signal.append(examples.copy())
                    concat_ch_examples_features.append(features)

                concat_ch_examples_signal = np.concatenate(concat_ch_examples_signal, axis=1)
                concat_ch_examples_features = np.concatenate(concat_ch_examples_features, axis=1)

                complete_examples_signal.append(concat_ch_examples_signal)
                complete_examples_features.append(concat_ch_examples_features)
                complete_labels.append(labels_)
                lt = len(times_)
                metadata_ = np.concatenate(
                    [pureness, times_, np.repeat(subject, lt)[:,None], np.repeat(session, lt)[:,None]],
                    axis=1
                )
                complete_metadata.append(metadata_)

        complete_examples_signal = np.concatenate(complete_examples_signal)
        complete_examples_features = np.concatenate(complete_examples_features)
        complete_labels = np.concatenate(complete_labels)
        complete_metadata = np.concatenate(complete_metadata)
        
        return complete_examples_signal, complete_examples_features, complete_labels, complete_metadata
    
    def __len__(self):
        return self.complete_examples_signal.shape[0]
    
    def __getitem__(self, idx):
        return {
            'signal': self.complete_examples_signal[idx,:],
            'features': self.complete_examples_features[idx,:],
            'label': self.complete_labels[idx,:],
            'metadata': self.complete_metadata[idx,:]
        }

    def get_X_signal(self):
        return self.complete_examples_signal

    def get_X_features(self):
        return self.complete_examples_features

    def get_Y(self):
        return self.complete_labels

    def get_metadata(self):
        return self.complete_metadata


Para levantar los datasets desde txt o csv:
- Este notebook, o el notebook que usen que tenga la clase arriba definida, tiene que estar en el mismo directorio que el archivo utils.py, que tiene algunas funciones usadas en los métodos de la clase.
- Descarguen el .zip de la base de datos de nuevo y extraiganlos a un directorio llamado <path al repo en su máquina>/MentoriaBCI/Database/
- Definan el csvs_path acorde al directorio anterior:
    - csvs_path = '<PATH AL REPO>/MetoriaBCI/Database' ej: '/home/joaquin/Desktop/MentoriaDiploDatos/MetoriaBCI/Database'

In [4]:
csvs_path = '../Database'

### A) b) Estudiar cómo varía el número de ejemplos en el dataset y la dimensión de cada dato según la variación de la ventana de tiempo seleccionada y el criterio de solapamiento.

In [13]:
dataset1 = BCIDataset(csvs_path, subject = "AA", session = "0")

print('Dimensiones de ventanas de dataset1:', dataset1.get_X_signal().shape)
print('Dimensiones de features de dataset1:', dataset1.get_X_features().shape)

dataset2 = BCIDataset(csvs_path, subject = "AA", session = "0", window_size = 450)

print('Dimensiones de ventanas de dataset2:', dataset2.get_X_signal().shape)
print('Dimensiones de features de dataset2:', dataset2.get_X_features().shape)

dataset3 = BCIDataset(csvs_path, subject = "AA", session = "0", overlapping_fraction=1/4)

print('Dimensiones de ventanas de dataset3:', dataset3.get_X_signal().shape)
print('Dimensiones de features de dataset3:', dataset3.get_X_features().shape)

dataset4 = BCIDataset(csvs_path, subject = "AA", session = "0", window_size=1800, overlapping_fraction=1/15)

print('Dimensiones de ventanas de dataset4:', dataset4.get_X_signal().shape)
print('Dimensiones de features de dataset4:', dataset4.get_X_features().shape)

Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset1: (151, 3600)
Dimensiones de features de dataset1: (151, 1804)
Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset2: (304, 1800)
Dimensiones de features de dataset2: (304, 904)
Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset3: (201, 3600)
Dimensiones de features de dataset3: (201, 1804)
Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset4: (361, 7200)
Dimensiones de features de dataset4: (361, 3604)


### B) c) En adición a la serie temporal cruda -”complete_examples_signal” de BCIDataset- (concatenada o no a lo largo de los canales, según su elección), defina una estrategia de extracción de atributos en el dominio de tiempo que opere sobre la serie cruda.

In [27]:
dataset5 = BCIDataset(csvs_path, subject = "AA", session = "0", feature_extractor = filtered_fft_features)

print('Dimensiones de ventanas de dataset5:', dataset5.get_X_signal().shape)
print('Dimensiones de features de dataset5:', dataset5.get_X_features().shape)

def sum400(signal_matrix):
    return signal_matrix + 400

dataset6 = BCIDataset(csvs_path, subject = "AA", session = "0", feature_extractor = sum400)

print('primer fila de dataset6:', dataset6.get_X_signal()[0,:5])
print()
print('primer fila de dataset6:', dataset6.get_X_features()[0,:5])

def mean_and_newcolumn(signal_matrix):
    means = signal_matrix.mean(axis=1)[:, np.newaxis]
    means = np.hstack([means, np.zeros(means.shape)])
    return means

dataset7 = BCIDataset(csvs_path, subject = "AA", session = "0", feature_extractor = mean_and_newcolumn)

print('Dimensiones de ventanas de dataset7:', dataset7.get_X_signal().shape)
print('Dimensiones de features de dataset7:', dataset7.get_X_features().shape)

print('primer fila de dataset7:', dataset7.get_X_signal()[0,:5])
print()
print('primer fila de dataset7:', dataset7.get_X_features()[0,:])

Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset5: (151, 3600)
Dimensiones de features de dataset5: (151, 1804)
Processing subject: AA - session: 0...
primer fila de dataset6: [-1.86 10.77 87.61 83.04  8.07]

primer fila de dataset6: [398.14 410.77 487.61 483.04 408.07]
Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset7: (151, 3600)
Dimensiones de features de dataset7: (151, 8)
primer fila de dataset7: [-1.86 10.77 87.61 83.04  8.07]

primer fila de dataset7: [ 51.51517778   0.          32.19205556   0.          54.17357778
   0.         -33.06123333   0.        ]


### C) a) Usando BCIDataset, con el extractor de features básico de fft, o el procesador de datos que desee, genere el dataset de ejemplos utilizado como atributos el espectrograma de potencia. 

In [40]:
pos_125 = 56 #cambiaria si cambia window_size
pos_165 = 74 #cambiaria si cambia window_size
window_size = 900

def super_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    fft = np.fft.rfft(signal_matrix)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    
    interest_freq_125 = Sxx[:, pos_125:pos_125+1]
    interest_freq_165 = Sxx[:, pos_165:pos_165+1]
    interest_freq = np.hstack([interest_freq_125, interest_freq_165])
    
    return interest_freq

dataset8 = BCIDataset(csvs_path, subject = "AA", session = "0", feature_extractor = super_fft_features)

print('Dimensiones de ventanas de dataset8:', dataset8.get_X_signal().shape)
print('Dimensiones de features de dataset8:', dataset8.get_X_features().shape)

print('primer fila de dataset1:', dataset1.get_X_features()[0,50:80])
print()
print('primer fila de dataset8:', dataset8.get_X_features()[0,:])

Processing subject: AA - session: 0...
Dimensiones de ventanas de dataset8: (151, 3600)
Dimensiones de features de dataset8: (151, 8)
primer fila de dataset1: [4.19762044 8.4187574  0.49655619 7.44543761 1.19898483 1.23263297
 5.16148451 0.72575445 1.42149166 3.04204042 2.70376405 1.31435521
 1.87550838 1.12606688 0.01666563 1.0739737  1.84534265 0.92770828
 0.04823913 0.70581164 0.29577067 3.6070825  3.41152378 2.39843848
 0.27087759 0.4858284  1.57678644 0.5845709  2.5801894  0.23302799]

primer fila de dataset8: [5.16148451 0.27087759 1.60691267 0.6609744  4.73681774 0.52951126
 2.80698654 0.42239365]
